In [6]:
import pandas as pd
import os
from pathlib import Path
import re

In [2]:
df = pd.read_csv('./original_0510.csv', encoding='utf-8-sig')

In [3]:
df.head()

,url,contents,title,source_file,date,link,제목,날짜,본문,댓글,URL
0,https://blog.naver.com/srhymin/223091161466,저희 아기는 신생아때부터\n청각과 촉각이 예민해\n잠도 깊게 못 잘 뿐더러\n잠투정...,NaN,아기 잠깨 취합.csv,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://blog.naver.com/sukm83/223091879514,정신없는 주방한켠\n뒤죽박죽 알 수 없는 용기에\n양념과 소스가 담겨져 있기에\n주...,NaN,아기 잠깨 취합.csv,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://blog.naver.com/dldptmf5134/223096875782,안녕하세요 지유맘입니다!\n오늘은 지유가 이유식과 유아식 그 중간즈음 토핑식을 할 ...,NaN,아기 잠깨 취합.csv,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://blog.naver.com/lia1100/223095799488,안녕하세요\n라온엄마에요\n오늘은 조금 특이한 아기 반찬을 준비했어요\n방풍나물을 ...,NaN,아기 잠깨 취합.csv,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://blog.naver.com/perfect_aj/223095824533,6~12개월 아기의 1일 권장 물섭취량은\n220ml정도라고 한다.\n음 우래기는....,NaN,아기 잠깨 취합.csv,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88826 entries, 0 to 88825
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   url          81333 non-null  object 
 1   contents     88824 non-null  object 
 2   title        30501 non-null  object 
 3   source_file  88826 non-null  object 
 4   date         7180 non-null   object 
 5   link         7180 non-null   object 
 6   제목           313 non-null    object 
 7   날짜           313 non-null    float64
 8   본문           313 non-null    object 
 9   댓글           311 non-null    object 
 10  URL          313 non-null    object 
dtypes: float64(1), object(10)
memory usage: 7.5+ MB


In [5]:
df['source_file'].value_counts()

source_file
아기 못자 취합.csv                                    22878
네이버블로그_낮잠 + 육아_230501-231120.csv          19551
아기 잠깨 취합.csv                                    17387
네이버블로그_낮잠 + 아기_230501-231130.csv            6504
DX_blog_낮잠+육아_20240501-20250501.csv                      3975
DX_blog_낮잠+아기_20231201-20240430.csv                3961
DX_blog_낮잠+아기_20240501-20250501.csv                3542
DX_blog_낮잠+육아_20231121-20240430.csv                      3535
맘스홀릭_낮잠+육아_본문포함_최종.csv           2342
맘스홀릭_아기+깨_본문포함.csv                      1450
맘스홀릭_아기+자_본문포함.csv                      1342
맘스홀릭_낮잠+아기_본문포함_최종.csv            1247
맘스홀릭_아기 + 못자_18중단_본문포함.csv         799
아기+잠깨_블라인드_웹크롤링_20230501이후.csv        127
낮잠+아기_블라인드_웹크롤링_20230501이후.csv       107
아기+못자_블라인드_웹크롤링_20230501이후.csv        

In [12]:
# source_file 매핑 정의
# {기존 source_file 값 : (새로운 source 값, 새로운 search_words 값)} 형태의 딕셔너리
source_file_mapping = {
    '아기 못자 취합.csv': ('블로그', '아기+못자'),
    '네이버블로그_낮잠 + 육아_230501-231120.csv': ('블로그', '낮잠+육아'),
    '아기 잠깨 취합.csv': ('블로그', '아기+잠깨'),
    '네이버블로그_낮잠 + 아기_230501-231130.csv': ('블로그', '낮잠+아기'),
    'DX_blog_낮잠+육아_20240501-20250501.csv': ('블로그', '낮잠+육아'),
    'DX_blog_낮잠+아기_20231201-20240430.csv': ('블로그', '낮잠+아기'),
    'DX_blog_낮잠+아기_20240501-20250501.csv': ('블로그', '낮잠+아기'),
    'DX_blog_낮잠+육아_20231121-20240430.csv': ('블로그', '낮잠+육아'),
    '맘스홀릭_낮잠+육아_본문포함_최종.csv': ('카페', '낮잠+육아'),
    '맘스홀릭_아기+깨_본문포함.csv': ('카페', '아기+깨'),
    '맘스홀릭_아기+자_본문포함.csv': ('카페', '아기+자'),
    '맘스홀릭_낮잠+아기_본문포함_최종.csv': ('카페', '낮잠+아기'),
    '맘스홀릭_아기 + 못자_18중단_본문포함.csv': ('카페', '아기+못자'),
    '아기+잠깨_블라인드_웹크롤링_20230501이후.csv': ('블라인드', '아기+잠깨'),
    '낮잠+아기_블라인드_웹크롤링_20230501이후.csv': ('블라인드', '낮잠+아기'),
    '아기+못자_블라인드_웹크롤링_20230501이후.csv': ('블라인드', '아기+못자'),
    '낮잠+육아_블라인드_웹크롤링_20230501이후.csv': ('블라인드', '낮잠+육아')
}

In [13]:
def get_source_and_search_words(filename):
    return source_file_mapping.get(filename, ('알수없음', '알수없음')) # 매핑에 없으면 '알수없음'

# 'source_file' 칼럼을 기반으로 'source'와 'search_words' 칼럼 생성
# apply의 결과가 튜플이므로, 이를 pd.Series로 변환하여 두 칼럼에 할당
df[['source', 'search_words']] = df['source_file'].apply(
    lambda x: pd.Series(get_source_and_search_words(x))
)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88826 entries, 0 to 88825
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   url           81333 non-null  object 
 1   contents      88824 non-null  object 
 2   title         30501 non-null  object 
 3   source_file   88826 non-null  object 
 4   date          7180 non-null   object 
 5   link          7180 non-null   object 
 6   제목            313 non-null    object 
 7   날짜            313 non-null    float64
 8   본문            313 non-null    object 
 9   댓글            311 non-null    object 
 10  URL           313 non-null    object 
 11  source        88826 non-null  object 
 12  search_words  88826 non-null  object 
dtypes: float64(1), object(12)
memory usage: 8.8+ MB


In [15]:
df.head()

,url,contents,title,source_file,date,link,제목,날짜,본문,댓글,URL,source,search_words
0,https://blog.naver.com/srhymin/223091161466,저희 아기는 신생아때부터\n청각과 촉각이 예민해\n잠도 깊게 못 잘 뿐더러\n잠투정...,NaN,아기 잠깨 취합.csv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,블로그,아기+잠깨
1,https://blog.naver.com/sukm83/223091879514,정신없는 주방한켠\n뒤죽박죽 알 수 없는 용기에\n양념과 소스가 담겨져 있기에\n주...,NaN,아기 잠깨 취합.csv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,블로그,아기+잠깨
2,https://blog.naver.com/dldptmf5134/223096875782,안녕하세요 지유맘입니다!\n오늘은 지유가 이유식과 유아식 그 중간즈음 토핑식을 할 ...,NaN,아기 잠깨 취합.csv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,블로그,아기+잠깨
3,https://blog.naver.com/lia1100/223095799488,안녕하세요\n라온엄마에요\n오늘은 조금 특이한 아기 반찬을 준비했어요\n방풍나물을 ...,NaN,아기 잠깨 취합.csv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,블로그,아기+잠깨
4,https://blog.naver.com/perfect_aj/223095824533,6~12개월 아기의 1일 권장 물섭취량은\n220ml정도라고 한다.\n음 우래기는....,NaN,아기 잠깨 취합.csv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,블로그,아기+잠깨


In [16]:
final_columns_ordered = ['source', 'search_words', 'url', 'title', 'contents']
df_modified = df[final_columns_ordered].copy() # .copy()를 사용하여 SettingWithCopyWarning 방지

print("\n수정된 데이터프레임 정보:")
df_modified.info()
print("\n수정된 데이터프레임 샘플:")
print(df_modified.head())

print("\n새로 생성된 source 및 search_words 샘플:")
print(df_modified[['source', 'search_words']].head(20))


수정된 데이터프레임 정보:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88826 entries, 0 to 88825
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   source        88826 non-null  object
 1   search_words  88826 non-null  object
 2   url           81333 non-null  object
 3   title         30501 non-null  object
 4   contents      88824 non-null  object
dtypes: object(5)
memory usage: 3.4+ MB

수정된 데이터프레임 샘플:
  source search_words                                              url title  \
0    블로그        아기+잠깨      https://blog.naver.com/srhymin/223091161466   NaN   
1    블로그        아기+잠깨       https://blog.naver.com/sukm83/223091879514   NaN   
2    블로그        아기+잠깨  https://blog.naver.com/dldptmf5134/223096875782   NaN   
3    블로그        아기+잠깨      https://blog.naver.com/lia1100/223095799488   NaN   
4    블로그        아기+잠깨   https://blog.naver.com/perfect_aj/223095824533   NaN   

                                            c

In [17]:
for col in df_modified.select_dtypes(include=['object']).columns:
    df_modified[col] = df_modified[col].astype('string')

print("\nstring Dtype으로 변경 후 데이터프레임 정보:")
df_modified.info()


string Dtype으로 변경 후 데이터프레임 정보:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88826 entries, 0 to 88825
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   source        88826 non-null  string
 1   search_words  88826 non-null  string
 2   url           81333 non-null  string
 3   title         30501 non-null  string
 4   contents      88824 non-null  string
dtypes: string(5)
memory usage: 3.4 MB


In [18]:
print(f"결측치 제거 전 행 수: {len(df_modified)}")
print(f"'contents' 컬럼의 결측치 수: {df_modified['contents'].isna().sum()}")

결측치 제거 전 행 수: 88826
'contents' 컬럼의 결측치 수: 2


In [19]:
full_df = df_modified.dropna(subset=['contents'])

print(f"\n결측치 제거 후 행 수: {len(full_df)}")
print(f"제거된 행 수: {len(df_modified) - len(full_df)}")
print(f"'contents' 컬럼의 결측치 수 (제거 후): {full_df['contents'].isna().sum()}")


결측치 제거 후 행 수: 88824
제거된 행 수: 2
'contents' 컬럼의 결측치 수 (제거 후): 0


In [20]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 88824 entries, 0 to 88825
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   source        88824 non-null  string
 1   search_words  88824 non-null  string
 2   url           81332 non-null  string
 3   title         30500 non-null  string
 4   contents      88824 non-null  string
dtypes: string(5)
memory usage: 4.1 MB


In [21]:
full_df.to_csv('full_data_0510.csv', index=False, encoding='utf-8-sig')